In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=dcd980c997b37b4c006c712d20dcaffeac8a9efbd77eb095d75511a4d4e85f4a
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F3 '''
def F3(X):
    f = bn.F3()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.1246 ,random_state=0 , l1_ratio=0.8181, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 83.68 , C = 976.04 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_3_4000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_3_800Samples.csv').iloc[:,-1]
path = "train_latent_200D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_200D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.1566 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=974576, Mon Apr 13 17:26:44 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.263027478812454e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:26:44 2020)
final/bestever f-value = 2.263027e+04 2.263027e+04
incumbent solution: [0.89965053 0.92931904 0.27141967 0.71012415 0.37354154 0.58309305
 0.07618496 0.49905561 ...]
std deviations: [0.15060508 0.15058132 0.15061001 0.15066132 0.15055225 0.15056803
 0.15066176 0.150615   ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=896586, Mon Apr 13 17:26:44 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.232225886811443e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [22630.27478812] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22630.274788, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22630.274788, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Mon Apr 13 17:26:44 2020)
final/bestever f-value = 2.232226e+04 2.232226e+04
incumbent solution: [0.68148522 0.97533876 0.31486503 0.90962516 0.57436794 0.6422856
 0.02032399 0.49586464 ...]
std deviations: [0.15089609 0.15088843 0.15082835 0.15102392 0.15094891 0.15090642
 0.15093521 0.15093504 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=873716, Mon Apr 13 17:26:45 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.797767935789069e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 2.656286454415459e+04 1.0e+00 1.46e-01  1e-01  1e-01 0:00.1
    3     51 2.572781811344628e+04 1.0e+00 1.43e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [29748.15989465] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [30287.72485232] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   82   1394 -7.397559469363688e+06 1.3e+00 2.59e+00  3e+00  3e+00 0:03.1
  100   1700 -4.352231722138614e+07 1.4e+00 6.08e+00  6e+00  6e+00 0:03.8
  170   2890 -2.503774724891221e+10 1.4e+00 1.58e+02  2e+02  2e+02 0:06.5
termination on tolfacupx=1000.0 (Mon Apr 13 17:26:52 2020)
final/bestever f-value = -2.470113e+10 -2.503775e+10
incumbent solution: [ 609.40935995  205.68850221  377.42334439 -312.61276845  735.35535172
   -8.64220022 -928.82223819   12.1664663  ...]
std deviations: [155.72693646 156.1700423  156.20906399 156.55072365 156.9312343
 155.54297171 155.8603056  156.16306454 ...]
Run : 1
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=864426, Mon Apr 13 17:26:52 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.263027478812454e+04 1.0e+00 1.50e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:26:52 2020)
final/bestever f-value = 2.263027e+04 2.263027e+04
incumbent solution: [ 0.03602977  0.42803165  0.7

...

   81   1377 -3.820250057779761e+06 1.3e+00 1.98e+00  2e+00  2e+00 0:03.1
  100   1700 -2.086420588697181e+07 1.3e+00 4.87e+00  5e+00  5e+00 0:03.8
  172   2924 -2.016978973301605e+10 1.4e+00 1.58e+02  2e+02  2e+02 0:06.6
termination on tolfacupx=1000.0 (Mon Apr 13 17:30:39 2020)
final/bestever f-value = -1.991794e+10 -2.026994e+10
incumbent solution: [ 715.50070548   30.19063429  368.36863786 -711.70973887  797.06643325
   55.06340932 -329.4533863    79.32844822 ...]
std deviations: [154.79370983 156.44562034 156.20684251 155.60305899 159.36300483
 156.93903359 155.24288873 154.45739051 ...]
Run : 29
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=930846, Mon Apr 13 17:30:39 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.263027478812454e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 17:30:39 2020)
final/bestever f-value = 2.263027e+04 2.263027e+04
incumbent solution: [0.03164509 0.74843914 0.75

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [22630.27478812] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22630.274788, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=22630.274788, sigma=1.51e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

termination on tolfun=1e-11 (Mon Apr 13 17:30:39 2020)
final/bestever f-value = 2.232226e+04 2.232226e+04
incumbent solution: [0.12734528 0.85230062 0.73745679 0.86456733 0.26845183 0.13878786
 0.33155057 0.52131073 ...]
std deviations: [0.15042642 0.15041449 0.15040244 0.15049117 0.15041576 0.15046555
 0.15053176 0.15040349 ...]
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=951078, Mon Apr 13 17:30:39 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 2.718977717979986e+04 1.0e+00 1.51e-01  2e-01  2e-01 0:00.0
    2     34 2.555602019892050e+04 1.0e+00 1.47e-01  1e-01  1e-01 0:00.1
    3     51 2.476983062503922e+04 1.0e+00 1.44e-01  1e-01  1e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [30140.02958221] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [27977.70521845] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   81   1377 -3.188595371651447e+06 1.3e+00 2.05e+00  2e+00  2e+00 0:03.1
  100   1700 -1.893066239072254e+07 1.3e+00 5.35e+00  5e+00  5e+00 0:03.9
  183   3111 -2.226294758455680e+10 1.4e+00 1.62e+02  2e+02  2e+02 0:07.0
termination on tolfacupx=1000.0 (Mon Apr 13 17:30:47 2020)
final/bestever f-value = -2.128771e+10 -2.226295e+10
incumbent solution: [525.926229   469.1899807  125.62159787  62.86682737 235.0463542
  -6.94191002 838.32477494 240.47102169 ...]
std deviations: [159.28249556 158.8658803  160.07226871 158.87648419 158.73979994
 158.57662028 159.28626588 162.56923173 ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F3_200_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F3(X_Values.iloc[i,:100])
    SVM_Fun [i] = F3(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F3(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([3910.19467739]), count=array([1])) 242.30653782551005
SVM
ModeResult(mode=array([3758.73521669]), count=array([1])) 275.05267676425933
ELN
ModeResult(mode=array([1.6970121e+33]), count=array([1])) 2.302584496262571e+53
